In [ ]:
import speech_recognition as sr
import time
import datetime
import os
import random
import json
import glob
import re
import pickle
from difflib import SequenceMatcher
from TTS.api import TTS
import simpleaudio as sa
import pygame
from mutagen.mp3 import MP3
from mutagen.easyid3 import EasyID3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import requests

# ----------------------------
# Config
# ----------------------------
VOICE_MODEL = "tts_models/en/ljspeech/tacotron2-DDC"
MUSIC_FOLDER = r"C:\Users\DIYOR\Music"  # change to your music folder
WAKE_WORD = "alex"
CONTINUOUS_MODE = False  # False = wake-word required; True = always listen
INTENT_MODEL_FILE = "intent_model.pkl"
INTENT_DATA_FILE = "intent_training.json"
LIGHT_URL = "http://172.20.10.2:80/toggle"  # your device endpoint

# ----------------------------
# Initialize STT & Audio
# ----------------------------
recognizer = sr.Recognizer()
mic = sr.Microphone()
pygame.mixer.init()
pygame.mixer.music.set_volume(0.7)  # default volume

# ----------------------------
# Load TTS
# ----------------------------
print("🔊 Loading TTS voice model...")
tts = TTS(model_name=VOICE_MODEL, progress_bar=False, gpu=True)
print("✅ Voice model loaded.")

# ----------------------------
# Load Training Data & Train Classifier
# ----------------------------
with open(INTENT_DATA_FILE, "r", encoding="utf-8") as f:
    training_data = json.load(f)

def preprocess(text: str) -> str:
    text = (text or "").lower().strip()
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

texts = [preprocess(item.get("text", "")) for item in training_data]
labels = [item.get("intent", "") for item in training_data]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

model = LogisticRegression(max_iter=2000)
model.fit(X, labels)

pickle.dump((model, vectorizer), open(INTENT_MODEL_FILE, "wb"))
print("✅ Intent classifier trained and saved.")

# ----------------------------
# Helper Functions
# ----------------------------
def speak(text: str):
    try:
        print("🤖 Alex says:", text)
        file_name = "alex_speak.wav"
        tts.tts_to_file(text=text, file_path=file_name)
        wave_obj = sa.WaveObject.from_wave_file(file_name)
        play_obj = wave_obj.play()
        play_obj.wait_done()
    except Exception as e:
        print("TTS error:", e)

def listen(seconds=5):
    """Listen and return recognized text. Behavior depends on CONTINUOUS_MODE."""
    with mic as source:
        try:
            recognizer.adjust_for_ambient_noise(source, duration=0.3)
            audio = recognizer.listen(source, timeout=seconds, phrase_time_limit=seconds)
            text = recognizer.recognize_google(audio, language="en-US").lower()
            print("🗣 You said:", text)
            if CONTINUOUS_MODE:
                return text
            if WAKE_WORD in text:
                return text
            return ""
        except sr.WaitTimeoutError:
            return ""
        except sr.UnknownValueError:
            return ""
        except Exception as e:
            print("Listening error:", e)
            return ""

def remove_wake_word(text: str) -> str:
    text = preprocess(text)
    if text.startswith(WAKE_WORD):
        return text[len(WAKE_WORD):].strip()
    if f" {WAKE_WORD} " in text:
        return text.split(WAKE_WORD, 1)[1].strip()
    return text

# ----------------------------
# Intent Matching
# ----------------------------
def match_command(user_input: str):
    user_input_clean = remove_wake_word(user_input)
    if not user_input_clean:
        return None, 0.0

    X_input = vectorizer.transform([user_input_clean])
    intent_pred = model.predict(X_input)[0]
    proba = model.predict_proba(X_input)[0]
    confidence = float(max(proba))

    if confidence < 0.5:
        best_score = 0.0
        best_command = None
        for item in training_data:
            score = SequenceMatcher(None, user_input_clean, preprocess(item.get("text", ""))).ratio()
            if score > best_score:
                best_score = score
                best_command = item
        if best_score > 0.6:
            return best_command.get("intent"), best_score
        return None, 0.0

    return intent_pred, confidence

# ----------------------------
# Device & Music States + Controls
# ----------------------------
device_states = {"light": False, "music": False}
current_song_index = -1
music_files_list = []
current_playing_song = None

def load_music_files():
    global music_files_list
    music_files_list = glob.glob(os.path.join(MUSIC_FOLDER, "*.mp3"))

def play_music():
    """Speak the song name and artist first, then play music"""
    global current_song_index, current_playing_song
    if not music_files_list:
        load_music_files()
    if not music_files_list:
        speak("No music found.")
        device_states["music"] = False
        return

    current_song_index = (current_song_index + 1) % len(music_files_list)
    selected_song = music_files_list[current_song_index]
    current_playing_song = selected_song
    song_name = os.path.basename(selected_song).replace(".mp3", "")

    # Speak title and artist first
    try:
        audio_meta = MP3(selected_song, ID3=EasyID3)
        title = audio_meta.get("title", [song_name])[0]
        artist = audio_meta.get("artist", ["Unknown Artist"])[0]
        speak(f"Playing {title} by {artist}")
    except Exception:
        speak(f"Playing {song_name}")

    # Play music after TTS
    try:
        pygame.mixer.music.load(selected_song)
        pygame.mixer.music.play()
        device_states["music"] = True
    except Exception as e:
        print("Playback error:", e)
        device_states["music"] = False

def stop_music():
    try:
        if pygame.mixer.music.get_busy():
            pygame.mixer.music.stop()
            device_states["music"] = False
            speak("Music stopped.")
        else:
            speak("No music is playing.")
    except Exception as e:
        print("Stop music error:", e)

def pause_music():
    try:
        if pygame.mixer.music.get_busy():
            pygame.mixer.music.pause()
            device_states["music"] = False
            speak("Music paused.")
        else:
            speak("Nothing to pause.")
    except Exception as e:
        print("Pause error:", e)

def resume_music():
    try:
        pygame.mixer.music.unpause()
        device_states["music"] = True
        speak("Resuming music.")
    except Exception as e:
        print("Resume error:", e)

def next_music():
    if not music_files_list:
        load_music_files()
    if not music_files_list:
        speak("No music found.")
        return
    try:
        if pygame.mixer.music.get_busy():
            pygame.mixer.music.stop()
        play_music()
    except Exception as e:
        print("Next music error:", e)

def set_volume(value: float):
    try:
        v = max(0.0, min(1.0, float(value)))
        pygame.mixer.music.set_volume(v)
    except Exception as e:
        print("Set volume error:", e)

def change_volume(delta: float):
    cur = pygame.mixer.music.get_volume()
    set_volume(cur + delta)

def extract_first_number(text: str):
    nums = re.findall(r"\d+", text)
    if not nums:
        return None
    try:
        return int(nums[0])
    except:
        return None

def enable_continuous_mode():
    global CONTINUOUS_MODE
    CONTINUOUS_MODE = True
    speak("Continuous listening enabled.")

def disable_continuous_mode():
    global CONTINUOUS_MODE
    CONTINUOUS_MODE = False
    speak("Continuous listening disabled. Say Alex first for commands.")

# ----------------------------
# Joke / Time / Date
# ----------------------------
def tell_joke():
    jokes = [
        "Why did the computer catch a cold? Bad Windows update!",
        "Why do programmers prefer dark mode? Light attracts bugs!",
        "What's a computer's favorite snack? Microchips!"
    ]
    return random.choice(jokes)

def get_time():
    now = datetime.datetime.now()
    return now.strftime("It's %I:%M %p.")

def get_date():
    today = datetime.date.today()
    return today.strftime("Today is %B %d, %Y.")

# ----------------------------
# Core Command Handler
# ----------------------------
def handle_command(user_input: str):
    user_input = user_input or ""
    intent, confidence = match_command(user_input)
    user_clean = remove_wake_word(user_input)

    # Continuous mode toggles
    if "listen continuously" in user_clean or "start continuous" in user_clean or "enable continuous" in user_clean:
        enable_continuous_mode()
        return
    if "stop listening" in user_clean or "stop continuous" in user_clean or "disable continuous" in user_clean:
        disable_continuous_mode()
        return

    if not intent:
        speak("Sorry, I didn't understand that.")
        return

    print(f"✅ Recognized intent '{intent}' ({confidence*100:.1f}% confidence)")

    # MUSIC
    if intent == "play_music":
        play_music()
        return
    if intent == "next_music":
        next_music()
        return
    if intent == "stop_music":
        stop_music()
        return
    if intent == "pause_music":
        pause_music()
        return
    if intent == "resume_music":
        resume_music()
        return

    # VOLUME
    if intent == "volume_up":
        change_volume(0.1)
        speak("Volume increased.")
        return
    if intent == "volume_down":
        change_volume(-0.1)
        speak("Volume decreased.")
        return
    if intent == "set_volume":
        num = extract_first_number(user_input)
        if num is None:
            speak("What volume percent should I set?")
            return
        pct = max(0, min(100, num))
        set_volume(pct / 100.0)
        #speak(f"Volume set to {pct} percent.")
        return

    # LIGHTS
    if intent == "light_on":
        device_states["light"] = True
        try:
            requests.post(LIGHT_URL, data="ON")
        except Exception as e:
            print("Light request failed:", e)
        speak("Turning on the lights.")
        return
    if intent == "light_off":
        device_states["light"] = False
        try:
            requests.post(LIGHT_URL, data="OFF")
        except Exception as e:
            print("Light request failed:", e)
        speak("Turning off the lights.")
        return

    # SYSTEM
    if intent == "notepad":
        speak("Opening Notepad.")
        try:
            os.system("notepad")
        except Exception as e:
            print("Notepad error:", e)
        return
    if intent == "sleep":
        speak("Going to sleep. Wake me by saying Alex.")
        time.sleep(1)
        return

    # INFO / MISC
    if intent == "tell_joke":
        speak(tell_joke())
        return
    if intent == "time":
        speak(get_time())
        return
    if intent == "date":
        speak(get_date())
        return
    if intent == "who_are_you":
        speak("I am Alex, your personal assistant.")
        return
    if intent == "greet":
        speak("Hello! How can I help you today?")
        return
    if intent == "how_are_you":
        speak("All good! How are you feeling today?")
        return

    # fallback
    speak("Sorry, I didn't understand that.")

# ----------------------------
# Main Loop
# ----------------------------
print("🎤 Alex is listening...")
load_music_files()

while True:
    try:
        user_input = listen(seconds=5)
        if not user_input:
            continue
        handle_command(user_input)
    except KeyboardInterrupt:
        print("\n👋 Shutting down Alex...")
        break
    except Exception as e:
        print("Error:", e)
        continue
